Building RAG [Smart Question and Answering System]

## Install Dependencies



In [1]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0
!pip install langgraph==0.1.1
!pip install langchain-chroma==0.1.1
!pip install langchain-google-genai

  Using cached langchain_openai-0.1.7-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.3/87.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

## Enter Open AI API Key or setup Gemini secret key in Colab

In [ ]:
#from getpass import getpass
#OPENAI_KEY = getpass('Enter Open AI API Key: ')
#import os
#os.environ['OPENAI_API_KEY'] = OPENAI_KEY
#from langchain_openai import OpenAIEmbeddings
#openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

Enter Open AI API Key: ··········


Mounting Drive to connect with Knowedge base

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load and Chunk Documents

In [3]:
import json
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter # Still imported, but not used for splitting
import os # Import os module for path operations
import glob # Import glob for pattern matching file paths
import gzip # Import gzip for handling gzipped files

docs = []

# 'knowledgebase'
folder_path = '/content/drive/MyDrive/knowledge_base_adani'
print(f"Attempting to load documents from dataset path: {folder_path}")

# Ensure the folder path exists
if not os.path.isdir(folder_path):
    print(f"Error: Folder '{folder_path}' not found. "
          f"Please ensure the dataset 'knowledgebase' is correctly added to your Kaggle Notebook "
          f"or mounted in your Colab environment.")
else:
    # Use glob to find all files in the specified folder.
    # You can add a pattern like '*.txt' or '*.md' if you only want specific file types.
    # For now, it will try to read all files.
    # Adjust the pattern if your documents have specific extensions (e.g., os.path.join(folder_path, '*.txt'))
    all_files_in_folder = glob.glob(os.path.join(folder_path, '*'))

    if not all_files_in_folder:
        print(f"No files found in '{folder_path}'. Please check the dataset contents.")

    # Loop through each file found in the folder
    for filepath in all_files_in_folder:
        # Check if it's actually a file (and not a sub-directory)
        if os.path.isfile(filepath):
            filename = os.path.basename(filepath) # Get just the file name from the full path
            try:
                with open(filepath, 'r', encoding='utf8') as f:
                    content = f.read()
                    # Create a Document object with the content and filename as metadata
                    docs.append(Document(page_content=content,
                                         metadata={'source': filename, 'path': filepath}))
                print(f"Successfully loaded document: {filename}")
            except Exception as e:
                print(f"Error loading document {filename} (path: {filepath}): {e}")
        # else:
        #     print(f"Skipping non-file item: {filepath}") # Uncomment to see skipped directories/symlinks


# --- Each document is now considered a single chunk ---
# Instead of splitting, we directly assign the loaded documents to chunked_docs.
# If you need to split the documents into smaller chunks, you can uncomment the following lines
# and adjust the chunk_size and chunk_overlap parameters.
# splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
# chunked_docs = splitter.split_documents(docs)
chunked_docs = docs


print(f"\nTotal original documents loaded: {len(docs)}")
print(f"Total chunks created: {len(chunked_docs)}") # This will now be the same as total documents loaded


Attempting to load documents from dataset path: /content/drive/MyDrive/knowledge_base_adani
Successfully loaded document: about_company.txt
Successfully loaded document: services_cloud.txt
Successfully loaded document: services_ai.txt
Successfully loaded document: case_study_finance.txt
Successfully loaded document: careers.txt
Successfully loaded document: genai_initiative.txt
Successfully loaded document: case_study_healthcare.txt
Successfully loaded document: vision_mission.txt
Successfully loaded document: leadership_team.txt
Successfully loaded document: training_services.txt
Successfully loaded document: cloud_security.txt
Successfully loaded document: awards.txt
Successfully loaded document: case_study_mfg.txt
Successfully loaded document: customer_testimonials.txt
Successfully loaded document: partners.txt
Successfully loaded document: contact_info.txt
Successfully loaded document: research_lab.txt
Successfully loaded document: internship_program.txt
Successfully loaded documen

In [4]:
len(chunked_docs)

30

In [5]:
chunked_docs[:5]

[Document(metadata={'source': 'about_company.txt', 'path': '/content/drive/MyDrive/knowledge_base_adani/about_company.txt'}, page_content='FutureTech Innovations, founded in 2010, specializes in AI, cloud computing, and data analytics for enterprises.'),
 Document(metadata={'source': 'services_cloud.txt', 'path': '/content/drive/MyDrive/knowledge_base_adani/services_cloud.txt'}, page_content='We provide cloud migration, optimization, and management services across AWS, Azure, and GCP.'),
 Document(metadata={'source': 'services_ai.txt', 'path': '/content/drive/MyDrive/knowledge_base_adani/services_ai.txt'}, page_content='FutureTech offers machine learning, NLP, computer vision, and generative AI consulting services.'),
 Document(metadata={'source': 'case_study_finance.txt', 'path': '/content/drive/MyDrive/knowledge_base_adani/case_study_finance.txt'}, page_content="A financial institution reduced loan processing time by 60% using FutureTech's AI-driven document processing."),
 Document(

### Create a Vector DB

In [6]:
# First, ensure you have the necessary library installed:
#!pip install langchain_chroma


from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from google.colab import userdata # Import userdata for accessing secrets

# --- Gemini API Key Setup ---
# It's highly recommended to load your API key from environment variables
# for security reasons, especially in production or shared environments like Colab.
# Set your API key from Colab secrets:
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    print("Google API key loaded from Colab secrets.")
except Exception as e:
    print(f"Error loading Google API key from Colab secrets: {e}")
    print("Please ensure your GOOGLE_API_KEY is set in Colab secrets.")


# Initialize the Gemini embedding model
# The model name 'models/embedding-001' is commonly used for embeddings.
# Ensure your GOOGLE_API_KEY environment variable is set or pass it directly.
gemini_embed_model = None # Initialize to None
try:
    gemini_embed_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    print("Gemini embedding model initialized successfully.")
except Exception as e:
    print(f"Error initializing Gemini embedding model: {e}")
    print("Please ensure your GOOGLE_API_KEY environment variable is set correctly and is valid.")
    # We will not exit here, but check if the model was initialized before using it.


# Assuming 'chunked_docs' is already defined from the previous steps
# create vector DB of docs and embeddings
if gemini_embed_model: # Check if the model was successfully initialized
    try:
        chroma_db = Chroma.from_documents(documents=chunked_docs,
                                     collection_name='rag_corporate_db',
                                     embedding=gemini_embed_model, # MODIFIED: Using Gemini embedding model
                                     # need to set the distance function to cosine else it uses euclidean by default
                                     # check https://docs.trychroma.com/guides#changing-the-distance-function
                                     collection_metadata={"hnsw:space": "cosine"},
                                     persist_directory="./corporate_db")

        print("\nChroma DB created/loaded successfully with Gemini embeddings.")
        print(f"Number of documents in Chroma DB: {chroma_db._collection.count()}")
    except Exception as e:
        print(f"Error creating Chroma DB: {e}")
else:
    print("\nSkipping Chroma DB creation because Gemini embedding model failed to initialize.")

Google API key loaded from Colab secrets.
Gemini embedding model initialized successfully.


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given



Chroma DB created/loaded successfully with Gemini embeddings.
Number of documents in Chroma DB: 30


### Setup a Vector Database Retriever

Here we use the following retrieval strategy:

- Similarity with Threshold Retrieval


### Similarity with Threshold Retrieval

We use cosine similarity here and retrieve the top 4 similar documents based on the user input query and also introduce a cutoff to not return any documents which are below a certain similarity threshold

In [7]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                        search_kwargs={"k": 4,
                                                                       "score_threshold": 0.3})

In [8]:
query = "About company, about Ethics, rewards, and products."
top4_docs = similarity_threshold_retriever.invoke(query)
top4_docs

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/ethics_board.txt', 'source': 'ethics_board.txt'}, page_content='We have an independent AI Ethics Board that governs sensitive use cases and ensures regulatory compliance.'),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/vision_mission.txt', 'source': 'vision_mission.txt'}, page_content='Our mission: Empower organizations through innovative technology. Vision: Ethical, sustainable AI leadership.'),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/careers.txt', 'source': 'careers.txt'}, page_content="We're hiring AI Engineers, Cloud Architects, and Data Scientists. Visit our Careers page!"),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/data_analytics_services.txt', 'source': 'data_analytics_services.txt'}, page_content='We offer descriptive, predictive, and prescriptive analytics services across industries.')]

In [9]:
query = "what are sustainability use case?"
top4_docs = similarity_threshold_retriever.invoke(query)
top4_docs

[Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/sustainability_initiatives.txt', 'source': 'sustainability_initiatives.txt'}, page_content='FutureTech is committed to green AI practices and reducing the carbon footprint of cloud workloads.'),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/corporate_social_responsibility.txt', 'source': 'corporate_social_responsibility.txt'}, page_content='FutureTech sponsors STEM education programs and AI for Good initiatives globally.'),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/ethics_board.txt', 'source': 'ethics_board.txt'}, page_content='We have an independent AI Ethics Board that governs sensitive use cases and ensures regulatory compliance.'),
 Document(metadata={'path': '/content/drive/MyDrive/knowledge_base_adani/partners.txt', 'source': 'partners.txt'}, page_content='Our strategic partners include AWS, Microsoft Azure, Google Cloud, and Nvidia.')]

## Create a Query Retrieval Grader

Here we will use an LLM itself to grade if any retrieved document is relevant to the given question - Answer will be either `yes` or `no`

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
#from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI # MODIFIED: Import Gemini chat model
import os


# Data model for LLM output format
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM for grading
# MODIFIED: Using ChatGoogleGenerativeAI with 'gemini-1.5-flash-latest' model
try:
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0) # MODIFIED model name
    structured_llm_grader = llm.with_structured_output(GradeDocuments)
    print("Gemini LLM grader initialized successfully.")
except Exception as e:
    print(f"Error initializing Gemini LLM grader: {e}")
    print("Please ensure your GOOGLE_API_KEY environment variable is set correctly.")
    # Exit or handle the error appropriately if the model cannot be initialized.



# Prompt template for grading
SYS_PROMPT = """You are an expert grader assessing relevance of a retrieved document to a user question.
                Follow these instructions for grading:
                  - If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
                  - Your grade should be either 'yes' or 'no' to indicate whether the document is relevant to the question or not.
             """
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Retrieved document:
                     {document}

                     User question:
                     {question}
                  """),
    ]
)

# Build grader chain
doc_grader = (grade_prompt
                  |
              structured_llm_grader)

Gemini LLM grader initialized successfully.


/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
query = "Tell me about the company and ethics and governance and reward system"
top4_docs = similarity_threshold_retriever.invoke(query)
for doc in top4_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

We have an independent AI Ethics Board that governs sensitive use cases and ensures regulatory compliance.
GRADE: binary_score='yes'

Our mission: Empower organizations through innovative technology. Vision: Ethical, sustainable AI leadership.
GRADE: binary_score='yes'

We're hiring AI Engineers, Cloud Architects, and Data Scientists. Visit our Careers page!
GRADE: binary_score='no'

We offer descriptive, predictive, and prescriptive analytics services across industries.
GRADE: binary_score='no'



In [19]:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
print("Google API key loaded from Colab secrets.")

query = "Tell me about the company and ethics and governance and salary"
top4_docs = similarity_threshold_retriever.invoke(query)
for doc in top4_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

Google API key loaded from Colab secrets.


GoogleGenerativeAIError: Error embedding content: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
query = "who won the cricket IPL match 2024?"
top4_docs = similarity_threshold_retriever.invoke(query)
for doc in top4_docs:
    print(doc.page_content)
    print('GRADE:', doc_grader.invoke({"question": query, "document": doc.page_content}))
    print()

We're hiring AI Engineers, Cloud Architects, and Data Scientists. Visit our Careers page!
GRADE: binary_score='no'

We're hiring AI Engineers, Cloud Architects, and Data Scientists. Visit our Careers page!
GRADE: binary_score='no'

FutureTech has won multiple AI Innovation Awards from TechWorld and CloudExpo.
GRADE: binary_score='no'

FutureTech has won multiple AI Innovation Awards from TechWorld and CloudExpo.
GRADE: binary_score='no'



## Build a QA RAG Chain

We will now connect our retriever to an LLM and build our QA RAG Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Create RAG prompt for response generation
prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know the answer.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer and to the point answer with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """
prompt_template = ChatPromptTemplate.from_template(prompt)

# Initialize connection with GPT-4o
#chatgpt = ChatOpenAI(model_name='gpt-4o', temperature=0)
chatgpt = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

# Used for separating context docs with new lines
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# create QA RAG chain
qa_rag_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter('question')
    }
      |
    prompt_template
      |
    chatgpt
      |
    StrOutputParser()
)

In [ ]:
query = "who is promoter of adani group?"
top4_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top4_docs, "question": query}
)
print(result)

The provided text does not contain any information about the promoter of the Adani Group.  Therefore, I don't know the answer.


In [ ]:
query = "Tell me about the company and ethics and governance and reward system"
top4_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top4_docs, "question": query}
)
print(result)

The provided text mentions the company's commitment to ethical and sustainable AI leadership.  Specifically, it highlights the existence of an independent AI Ethics Board responsible for overseeing sensitive AI applications and maintaining regulatory compliance.  No further details about the company itself, its overall ethics and governance structure, or its reward system are given.


In [ ]:
query = "which product you have launched on predictive ai. Let me know various cases studies or implementations?"
top4_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top4_docs, "question": query}
)
print(result)

Based on the provided text,  a predictive AI product offering AI recommendation engines has been launched.  A case study shows that retail clients experienced a 30% increase in personalized sales using these engines.  No other case studies or implementations are detailed in the given context.


## Create a Query Rephraser

We will now build a query rephraser which will use an LLM to rephrase the input user query into a better version which is optimized for web search

In [ ]:
# LLM for question rewriting
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)


# Prompt template for rewriting
SYS_PROMPT = """Act as a question re-writer and perform the following task:
                 - Convert the following input question to a better version that is optimized for web search.
                 - When re-writing, look at the input question and try to reason about the underlying semantic intent / meaning.
             """
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Here is the initial question:
                     {question}

                     Formulate an improved question.
                  """,
        ),
    ]
)
# Create rephraser chain
question_rewriter = (re_write_prompt
                        |
                       llm
                        |
                     StrOutputParser())

In [ ]:
query = "which are employee friendly policies"
question_rewriter.invoke({"question": query})

'Best employee-friendly company policies'

In [ ]:
query = "which product you have launched on predictive ai. Let me know various cases studies or implementations?"
query = question_rewriter.invoke({"question": query})
print(query)

top4_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": top4_docs, "question": query}
)
print(result)

Here are a few improved questions, each targeting slightly different aspects of the original query:

**Option 1 (Broadest):**

"Predictive AI case studies and implementations: Which products use this technology?"

**Option 2 (More specific, focusing on a particular company -  requires knowing the company name):**

"[Company Name] predictive AI products: case studies and implementations"  (Replace "[Company Name]" with the actual company name)


**Option 3 (Focuses on specific applications):**

"Predictive AI applications and case studies in [Industry/Area]" (Replace "[Industry/Area]" with a relevant field like healthcare, finance, etc.)


**Option 4 (If interested in open-source solutions):**

"Open-source predictive AI projects: case studies and implementations"


The improvements include:

* **Clarity:**  The original question is grammatically awkward. The revised options are clearer and more concise.
* **Keywords:**  The revised options use more precise keywords that search engines 

In [ ]:
!pip install pydantic<2

/bin/bash: line 1: 2: No such file or directory


In [ ]:
!pip install pydantic==1.10.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires langchain-core<0.4.0,>=0.3.68, but you have langchain-core 0.2.43 which is incompatible.
langchain-google-genai 2.1.7 requires pydantic<3,>=2, but you have pydantic 1.10.13 which is incompatible.
google-genai 1.24.0 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.13 which is incompatible.
gradio 5.31.0 requires pydantic<2.12,>=2.0, but you have pydantic 1.10.13 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4

In [ ]:
!pip install --upgrade langchain-core

  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langsmith-0.4.5-py3-none-any.whl.metadata (15 kB)
Using cached langchain_core-0.3.68-py3-none-any.whl (441 kB)
Using cached langsmith-0.4.5-py3-none-any.whl (367 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 0.1.1 requires langchain-core<0.3,>=0.2, but you have langchain-core 0.3.68 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.68 